## How others did it
- IEEE International Symposium on Biomedical Imaging 2021 challenge: Large-scale 3D Mitochondria Instance Segmentation
<img src="figures/isbi.png" style="width:80%"/>

## Multilayer perceptron
$\hat{y} = \mathbf{w}^\top\mathbf{x} + b$

## Loss function
- Dissimilarity measure between model prediction and target
- Squared error in regression problems: $l(\mathbf{w},b) = \frac{1}{2} (\hat{y} - y)^2$

## Optimization
- Finding parameters that minimize the loss function: $\mathbf{w}^{*},b^{*} = \underset{\mathbf{w},b}{\operatorname{argmin}} L(\mathbf{w},b)$
- Various flavors of gradient descent
- In a nutshell, parameters are updated in the direction of the negative gradient by step size $\eta$ (learning rate): $$(\mathbf{w},b) \gets (\mathbf{w},b) - \eta \nabla L(\mathbf{w},b)$$